# Visão geral dos dados coletados

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import sys


sys.path.append("../../")
sys.path.append("../../odbc")
import query_tools as qt
from connection import load_conn

## Linhas coletadas

### Quantas linhas de informação dos voos foram coletadas?

In [ ]:
q = """
SELECT count(*)
FROM flight.search
"""
qt.run_query(q)

### Em média quantas linhas novas são coletadas por dia?
Obs: considerando que o crawler coletaria 24h por dia
Uma boa estimativa é verificar qual a média de linhas coletadas por hora e multiplicar por 24h, vai ser uma estimativa melhor.

In [ ]:
# Media e desvio padrao do nº de linhas coletadas por hora
q = """
SELECT AVG(count) AS average_rows_por_hora, stddev(count) AS desvio_padrao_por_hora
FROM (
    SELECT COUNT(*) AS count
    FROM flight.search
    GROUP BY DATE_TRUNC('hour', "operationalSearchTime")
) subquery
"""
metricas_coleta_linhas_por_hora = qt.run_query(q)
metricas_coleta_linhas_por_hora

In [ ]:
estimativa_linhas_coletadas_por_dia = metricas_coleta_linhas_por_hora.loc[0, "average_rows_por_hora"] * 24
estimativa_linhas_coletadas_por_dia

## Memória

### Quantos Bytes tem o banco de dados?

In [ ]:
# Consulta para obter o tamanho do banco de dados em bytes
q = "SELECT pg_database_size('flight')"
bytes_df = qt.run_query(q)
bytes_df

### Quantos Gigabytes tem o banco de dados?

In [ ]:
bytes_ = bytes_df.loc[0, "pg_database_size"]
# Consulta para obter o tamanho do banco de dados em gigabytes 
q = f"SELECT pg_size_pretty({bytes_})"
qt.run_query(q)

## Coleta de dados

### Quais as datas e horários que temos coletados?

In [ ]:
q = """
SELECT DISTINCT DATE_TRUNC('hour', "operationalSearchTime") AS "operationalSearchTime"
FROM flight.search
"""

operationalSearchTime = qt.run_query(q)

In [ ]:
operationalSearchTime.head(1)

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(operationalSearchTime['operationalSearchTime'],
            [1] * len(operationalSearchTime))
plt.yticks([])  # Remove os valores no eixo y
plt.xlabel('Data e Hora')
plt.title('Datas e Horários Coletados')
plt.show()


### Qual a quantidade de voos capturados em cada coleta?

In [ ]:
q = """
SELECT DISTINCT DATE_TRUNC('hour', "operationalSearchTime") AS hora_coletada,
    count(*) AS linhas_coletadas
FROM flight.search
GROUP BY hora_coletada
"""
coleta = qt.run_query(q)

In [ ]:
coleta.hora_coletada.max(), coleta.hora_coletada.min()

In [ ]:
coleta.head(10).to_dict()

In [ ]:
# Definir intervalo de datas completo
start_date = coleta['hora_coletada'].min()
end_date = coleta['hora_coletada'].max()
date_range = pd.date_range(start=start_date, end=end_date, freq='H')

# Criar DataFrame com todas as horas do intervalo
complete_df = pd.DataFrame({'hora_coletada': date_range})

# Fazer merge dos DataFrames
coleta_completa = pd.merge(complete_df, coleta, on='hora_coletada', how='left')
coleta_completa['linhas_coletadas'] = coleta_completa['linhas_coletadas'].fillna(0)

# Ordenar DataFrame pelo horário
coleta_completa = coleta_completa.sort_values('hora_coletada')


In [ ]:
# Criando o gráfico
plt.figure(figsize=(15, 5))
plt.plot(coleta_completa["hora_coletada"], coleta_completa["linhas_coletadas"])
plt.xticks(rotation=90)

# Configurando as datas no eixo x
num_ticks = 60  # Número desejado de ticks no eixo x
date_range = pd.date_range(start=coleta_completa["hora_coletada"].min(), end=coleta_completa["hora_coletada"].max(), periods=num_ticks)
plt.xticks(date_range, rotation=90)

plt.xlabel('Data e Hora')
plt.ylabel('Linhas Coletadas')
plt.title('Quantidade de Linhas Coletadas por Data')
plt.show()


### Qual a maior janela de coleta consecutiva?

In [ ]:
coleta_completa.sort_values("hora_coletada", inplace=True, ignore_index=True)
display(coleta_completa.head())

teve_coleta = (coleta_completa.linhas_coletadas != 0).map(int)

In [ ]:
# Variáveis para acompanhar a sequência atual e a sequência máxima
numero_zeros_aceitaveis = 0
current_sequence = 0
max_sequence = 0
primeiro_valor_sequencia = True
menor_data = None
maior_data = None

contador_zeros = 0

# Percorrer a série
for indice, value in enumerate(teve_coleta):
    if value == 1:
        if primeiro_valor_sequencia:
            menor_data_sequencia_atual = coleta_completa.loc[indice, "hora_coletada"]
            primeiro_valor_sequencia = False

        current_sequence += 1
        contador_zeros = 0
        if current_sequence > max_sequence:
            max_sequence = current_sequence
            menor_data = menor_data_sequencia_atual
            maior_data = coleta_completa.loc[indice, "hora_coletada"]
    else:
        contador_zeros += 1
        
        if contador_zeros > numero_zeros_aceitaveis:
            current_sequence = 0
            primeiro_valor_sequencia = True

print("Maior sequência de 1's:", max_sequence, f"Isso equivale a {round(max_sequence / 24, 2)} dias")
print(f"Data de inicio da sequencia {menor_data} , data do fim da sequencia {maior_data}")

In [ ]:
# Conferindo o intervalo encontrado
numero_horas = len(pd.date_range(start=menor_data, end=maior_data, freq='H'))
numero_horas, numero_horas/24

## Voos únicos

### Quantos voos únicos foram coletados?

In [ ]:
q = """
SELECT count(*) as voos_unicos
FROM (SELECT DISTINCT "legId" FROM flight.fare) subquery
"""
qt.run_query(q)

### Em média quantos voos únicos são coletados por dia?


In [ ]:
q = """
SELECT AVG(voos_unicos) as media_voos_unicos
FROM (
    SELECT DATE_TRUNC('day', "operationalSearchTime") as search_day,
        count(*) AS voos_unicos
    FROM (SELECT DISTINCT F."legId", S."operationalSearchTime"
          FROM flight.fare F
          JOIN flight.search S ON F."searchId" = S."searchId"
          ) subquery
    GROUP BY search_day
) subquery2
"""

media_voos_unicos_dia = qt.run_query(q)
media_voos_unicos_dia

In [ ]:
media_voos_unicos_dia["media_voos_unicos"] = media_voos_unicos_dia["media_voos_unicos"].apply(lambda x: '{:,.2f}'.format(x))
media_voos_unicos_dia
# TEM ALGO MUITO ESTRANHO AQUI, VERIFICAR ISSO

## Cidade

In [ ]:
airport = qt.get_table("airport")
airport = airport[["airportCode", "city"]]

def merge_airport(df, column_merge):
    if isinstance(column_merge, str):
        df = df.merge(airport, how="left", left_on=column_merge, right_on="airportCode")
        del df["airportCode"]
    elif isinstance(column_merge, list):
        for index, column in enumerate(column_merge):
            old_column = "" if index == 0 else column_merge[(index-1)]
            df = df.merge(airport, how="left",
                          left_on=column,
                          right_on="airportCode",
                          suffixes=(old_column, f"_{column}"))
            del df["airportCode"]
    return df

### Quantos voos únicos saem de cada cidade?

In [ ]:
q = """
SELECT "originCode", count(*) as voos_unicos
FROM (
    SELECT DISTINCT "legId", "originCode"
    FROM flight.fare F
        JOIN flight.search S
        ON S."searchId" = F."searchId" 
) subquery
GROUP BY "originCode"
ORDER BY voos_unicos DESC
"""
voos_unicos_saida = qt.run_query(q)

In [ ]:
voos_unicos_saida

### Quantos voos únicos chegam em cada cidade?

In [ ]:
q = """
SELECT "destinationCode", count(*) as voos_unicos
FROM (
    SELECT DISTINCT "legId", "destinationCode"
    FROM flight.fare F
        JOIN flight.search S
        ON S."searchId" = F."searchId" 
) subquery
GROUP BY "destinationCode"
ORDER BY voos_unicos DESC
"""
voos_unicos_chegada = qt.run_query(q)

In [ ]:
voos_unicos_chegada = merge_airport(voos_unicos_chegada, "destinationCode")
voos_unicos_chegada

### Quantos voos únicos existem em cada trecho?

In [ ]:
q = """
SELECT "originCode", "destinationCode", count(*) as voos_unicos
FROM (
    SELECT DISTINCT "legId", "originCode", "destinationCode"
    FROM flight.fare F
        JOIN flight.search S
        ON S."searchId" = F."searchId" 
) subquery
GROUP BY  "originCode", "destinationCode"
ORDER BY voos_unicos DESC
"""
voos_unicos_trecho = qt.run_query(q)

In [ ]:
voos_unicos_trecho = merge_airport(voos_unicos_trecho, ["originCode", "destinationCode"])
voos_unicos_trecho

## Companias aéreas 

### Quais são as companhias aéreas coletadas?

In [ ]:
qt.get_table("airline")[["airlineCode", "airlineName"]].drop_duplicates()

### Quantos voos únicos cada companía fez?

In [ ]:
q = """
SELECT "airlineName", count(*) as voos_unicos
FROM (
    SELECT DISTINCT F."legId", A."airlineName"
    FROM flight.flight F
        LEFT JOIN flight.airline A
        ON A."airlineCode" = F."airlineCode"
) subquery
GROUP BY  "airlineName"
ORDER BY voos_unicos DESC
"""
voos_unicos_compania = qt.run_query(q)

In [ ]:
voos_unicos_compania